## Tratamento dos dados

### Importando bibliotecas

In [88]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
import pandas as pd

### Geração de dados sintéticos

In [89]:
def gerar_configuracao(mu_list, sigma_list, tamanhos, config_id):
    dfs = []
    for i, (mu, sigma2, n) in enumerate(zip(mu_list, sigma_list, tamanhos)):
        Sigma = np.diag(sigma2)
        data = np.random.multivariate_normal(mu, Sigma, n)
        df = pd.DataFrame(data, columns=["x1", "x2"])
        df["class"] = i + 1
        dfs.append(df)
    df_config = pd.concat(dfs, ignore_index=True)
    df_config["config"] = config_id
    return df_config

np.random.seed(42)  # reprodutibilidade

# -------------------------------
# Configuração 1
mu_1 = [[5, 0], [15, 5], [18, 14]]
sigma2_1 = [[81, 9], [9, 100], [25, 36]]
n1 = [200, 100, 50]
df1 = gerar_configuracao(mu_1, sigma2_1, n1, config_id=1)
l1 = "Classes elípticas de tamanhos diferentes"

# -------------------------------
# Configuração 2
mu_2 = [[0, 0], [30, 0], [12, 25]]
sigma2_2 = [[100, 100], [49, 49], [16, 16]]
n2 = [200, 100, 50]
df2 = gerar_configuracao(mu_2, sigma2_2, n2, config_id=2)
l2 = "Classes esféricas de tamanhos diferentes"

# -------------------------------
# Configuração 3
mu_3 = [[0, 0], [15, 5], [15, -5]]
sigma2_3 = [[100, 4], [100, 4], [100, 4]]
n3 = [100, 100, 100]
df3 = gerar_configuracao(mu_3, sigma2_3, n3, config_id=3)
l3 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 4
mu_4 = [[0, 0], [15, 0], [-15, 0]]
sigma2_4 = [[16, 16], [16, 16], [16, 16]]
n4 = [100, 100, 100]
df4 = gerar_configuracao(mu_4, sigma2_4, n4, config_id=4)
l4 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 5
mu_5 = [[5, 0], [15, 5], [10, -7], [3, 15]]
sigma2_5 = [[81, 9], [9, 100], [49, 16], [25, 25]]
n5 = [50, 50, 50, 50]
df5 = gerar_configuracao(mu_5, sigma2_5, n5, config_id=5)
l5 = "3 classes elípticas e 1 esférica"

# -------------------------------
# Configuração 6
mu_6 = [[5, 0], [15, 5], [12, -12], [7, 17]]
sigma2_6 = [[81, 9], [9, 100], [16, 16], [25, 25]]
n6 = [50, 50, 50, 50]
df6 = gerar_configuracao(mu_6, sigma2_6, n6, config_id=6)
l6 = "2 classes elípticas e 2 esféricas"

# -------------------------------
# Configuração 7
mu_7 = [[0, 0], [18, 0], [-18, 0], [0, -12]]
sigma2_7 = [[12, 12], [20, 20], [16, 16], [81, 20]]
n7 = [50, 50, 50, 50]
df7 = gerar_configuracao(mu_7, sigma2_7, n7, config_id=7)
l7 = "1 classe elíptica e 3 esféricas"

### Escolhendo e verificando um dos conjuntos gerados

In [90]:
df = df7
df.head()

,x1,x2,class,config
0,1.702362,1.973708,1,7
1,0.677169,-0.342410,1,7
2,1.510263,-8.771958,1,7
3,2.362758,0.437093,1,7
4,-0.770676,7.090539,1,7


### Verificando as classes

In [91]:
df["class"].unique()

array([1, 2, 3, 4])

### Verificando o nome exato das colunas

In [92]:
df.columns

Index(['x1', 'x2', 'class', 'config'], dtype='object')

### Retirando variáveis inúteis e transformando classes em números

In [93]:
df.drop("config", axis=1, inplace=True)
df["class"].replace({1: 0, 2: 1, 3: 2, 4: 3}, inplace=True)
df.head()

/tmp/ipykernel_1573/207842191.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["class"].replace({1: 0, 2: 1, 3: 2, 4: 3}, inplace=True)


,x1,x2,class
0,1.702362,1.973708,0
1,0.677169,-0.342410,0
2,1.510263,-8.771958,0
3,2.362758,0.437093,0
4,-0.770676,7.090539,0


### Verificando a corretude das classes

In [94]:
df["class"].unique()

array([0, 1, 2, 3])

### Armazenando as classes em uma variável separada

In [95]:
labels = df["class"].values
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3])

In [96]:
df.drop("class", axis=1, inplace=True)
df.head()

,x1,x2
0,1.702362,1.973708
1,0.677169,-0.342410
2,1.510263,-8.771958
3,2.362758,0.437093
4,-0.770676,7.090539


### Retirando a classe para isolar as variáveis

In [97]:
dados = df.to_numpy()
dados

array([[ 1.70236156e+00,  1.97370777e+00],
       [ 6.77168582e-01, -3.42409821e-01],
       [ 1.51026305e+00, -8.77195775e+00],
       [ 2.36275802e+00,  4.37093080e-01],
       [-7.70675649e-01,  7.09053864e+00],
       [-2.33804002e+00, -1.39629618e+00],
       [ 6.89894301e+00, -2.88234491e+00],
       [-1.90565524e+00, -5.09311007e-01],
       [ 2.91238968e+00,  7.19849001e-01],
       [-4.47778555e+00, -1.84348598e+00],
       [-2.10393747e+00, -2.67501547e-01],
       [ 1.47516571e+00,  1.44870723e+00],
       [-6.15228519e+00,  3.68559969e+00],
       [ 8.74924147e-01,  4.79615970e+00],
       [ 1.53874814e+00,  3.81509464e+00],
       [ 1.61616037e+00,  4.66346282e+00],
       [ 1.80912927e+00, -3.39188363e-01],
       [ 7.87194479e+00,  3.07971493e+00],
       [ 1.98750898e+00, -4.42124819e+00],
       [-4.46282850e+00,  1.01399617e+00],
       [ 5.04100216e-01, -2.12738297e+00],
       [ 4.88378397e-01,  5.50316511e+00],
       [ 2.40894484e+00, -3.95256942e+00],
       [-3.

## Clustering

### Inicialização da matriz de pertinência

A matriz de pertinência é inicializada aleatoriamente $u_{ik}(i=1,...c$ e $k=1,...,n)$ do objeto $k$ pertencente ao grupo $C_i$ tal que:
- $u_{ik} \in [0,1]$;
- $0 < \sum_{k=1}^nu_{ik} < n$;
- $\sum_{i=1}^cu_{ik} = 1$ para todo $k \in \Omega$.

In [98]:
def inicializacao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

### Atualização dos centroides

Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:

### $y_i = \frac{\sum_{k=1}^n(u_{ik})^mx_k}{\sum_{k=1}^n(u_{ik})^m}$

In [99]:
def atualizacao_centroides(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m # preparação dos graus de pertinência
    centroides = np.dot(matriz_pertinencia_m.T, dados) / np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True) # fórmula para o cálculo dos centroides
    return centroides

### Atualização da matriz de pertinência

Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:

### $u_{ik} = [\sum_{h=1}^c\{\frac{d(x_k,y_i)}{d(x_k,y_h)}\}^\frac{1}{m-1}]^{-1}$

onde

$d(x_k,y_i) = \sum_{j=1}^p(x_k^j-y_i^j)^2$

In [100]:
def atualizacao_matriz_pertinencia(dados, centroides, m):
    # dados[:, np.newaxis] - centroides cria uma matriz de diferenças entre os pontos de dados e os centroides
    # np.linalg.norm(..., axis=2) calcula a norma (distância euclidiana) das diferenças
    # ** 2 para a distância ser a quadrada
    matriz_distancias = np.linalg.norm(dados[:, np.newaxis] - centroides, axis=2) ** 2
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps) # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aaqui
    matriz_distancias_inversa = 1 / matriz_distancias
    potencia = 1 / (m-1)
    matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia/ np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    return matriz_pertinencia_atualizada

### Fuzzy C-Means

Ações:
1. Inicialização da matriz de pertinência
2. Atualização dos centroides
3. Atualização da matriz de pertinência

Critérios de parada:
1. Número máximo de iterações atingido
2. Pouca diferença (erro) entre as matrizes de pertinência de iterações consecutivas

In [101]:
def fcm(dados, num_clusters, m=2, max_iter=10**6, erro=1e-9):
    num_amostras = dados.shape[0]
    matriz_pertinencia = inicializacao_matriz_pertinencia(num_amostras, num_clusters)
    for _ in range(max_iter): # primeiro critério de parada
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, m)
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
            break
        matriz_pertinencia = nova_matriz_pertinencia
    return centroides, matriz_pertinencia

### Índice de Rand Ajustado (IRA)

In [102]:
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)

### Simulação de Monte Carlo

In [103]:
def simulacao_monte_carlo(dados, labels, num_clusters, num_trials):
    indices_rand = []
    for _ in range(num_trials):
        centroides, matriz_pertinencia = fcm(dados, num_clusters)
        predicted_labels = np.argmax(matriz_pertinencia, axis=1)
        #print(predicted_labels)
        idx_rand = indice_rand(labels, predicted_labels)
        indices_rand.append(idx_rand)
    mean_rand_index = np.mean(indices_rand)
    std_rand_index = np.std(indices_rand)
    return mean_rand_index, std_rand_index

### Definição de parâmetros e execução do método

In [104]:
num_clusters = 4
num_trials = 100
media_indice_rand, dp_indice_rand = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)

print(f"Monte Carlo FCM Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {media_indice_rand:.4f}")
print(f"Standard Deviation of Rand Index: {dp_indice_rand:.4f}")

Monte Carlo FCM Clustering Results (100 trials)
Mean Rand Index: 0.7653
Standard Deviation of Rand Index: 0.0000


### Resultados

Configuração 1:  
Mean Rand Index: 0.2943  
Standard Deviation of Rand Index: 0.0000  

Configuração 2:  
Mean Rand Index: 0.6129  
Standard Deviation of Rand Index: 0.0000  

Configuração 3:  
Mean Rand Index: 0.1285  
Standard Deviation of Rand Index: 0.0000  

Configuração 4:  
Mean Rand Index: 0.8470  
Standard Deviation of Rand Index: 0.0000  

Configuração 5:  
Mean Rand Index: 0.4291  
Standard Deviation of Rand Index: 0.0000  

Configuração 6:  
Mean Rand Index: 0.4814  
Standard Deviation of Rand Index: 0.0000  

Configuração 7:  
Mean Rand Index: 0.7653  
Standard Deviation of Rand Index: 0.0000